In [4]:
import torch
import torch.nn as nn
import pandas as pd
import pickle
from sklearn.model_selection import KFold, StratifiedKFold

In [5]:
import numpy as np

In [6]:
a = pd.read_csv('/home/aa0918358122/recommender_system/homework_03/tbrain_cc_training_48tags_hash_final.csv')

## User Transaction Data

In [7]:
'''
b = a[['chid', 'dt', 'shop_tag', 'txn_cnt', 'txn_amt',
       'domestic_offline_cnt', 'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
       'domestic_offline_amt_pct', 'domestic_online_amt_pct', 'overseas_offline_amt_pct', 'overseas_online_amt_pct',
       'masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam', 'gender_code', 'age', 'primary_card']]

b['shop_tag'] = b['shop_tag'].apply(lambda x: 49 if x == 'other' else int(x))
b['gender_code'] = b['gender_code'].fillna(2)

b[['masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'age', 'primary_card']] = b[['masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'age', 'primary_card']].fillna(0)
'''

useful_column = ['chid', 'dt', 'shop_tag', 'txn_cnt', 'txn_amt']
useful_column += [f'card_{i}_txn_amt_pct' for i in range(1, 15)] + ['card_other_txn_amt_pct']
b = a[useful_column]

In [8]:
pred_class = {2, 6, 10, 12, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48}

In [9]:
all_transactions = {}

In [10]:
for i, (k, v) in enumerate(b.groupby('chid')):
    curr_user_id = v['chid'].unique()[0]
    curr_user_df = v[useful_column]
    
    #curr_user_df = v[['chid', 'dt', 'shop_tag', 'txn_cnt', 'txn_amt',
    #                  'domestic_offline_cnt', 'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
    #                  'domestic_offline_amt_pct', 'domestic_online_amt_pct', 'overseas_offline_amt_pct', 'overseas_online_amt_pct',        
    #                  'masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam', 'gender_code', 'age', 'primary_card']]
    
    #curr_user_df['slam'] = curr_user_df['slam'].fillna(curr_user_df['slam'].median())
    #curr_user_df[['masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'age', 'primary_card']] = curr_user_df[['masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'age', 'primary_card']].fillna(curr_user_df[['masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'age', 'primary_card']].mode())
    
    if i % 100 == 0:
        print(f'user {i}: {curr_user_id}')
    
    user_transaction = []
    for k, v in curr_user_df.sort_values(['txn_amt'], ascending=False).groupby('dt'):
        curr_transaction = v.values.tolist()
        user_transaction.append(curr_transaction)
    
    all_transactions[curr_user_id] = user_transaction

user 0: 10000000
user 100: 10000100
user 200: 10000200
user 300: 10000300
user 400: 10000400
user 500: 10000500
user 600: 10000600
user 700: 10000700
user 800: 10000800
user 900: 10000900
user 1000: 10001000
user 1100: 10001100
user 1200: 10001200
user 1300: 10001300
user 1400: 10001400
user 1500: 10001500
user 1600: 10001600
user 1700: 10001700
user 1800: 10001800
user 1900: 10001900
user 2000: 10002000
user 2100: 10002100
user 2200: 10002200
user 2300: 10002300
user 2400: 10002400
user 2500: 10002500
user 2600: 10002600
user 2700: 10002700
user 2800: 10002800
user 2900: 10002900
user 3000: 10003000
user 3100: 10003100
user 3200: 10003200
user 3300: 10003300
user 3400: 10003400
user 3500: 10003500
user 3600: 10003600
user 3700: 10003700
user 3800: 10003800
user 3900: 10003900
user 4000: 10004000
user 4100: 10004100
user 4200: 10004200
user 4300: 10004300
user 4400: 10004400
user 4500: 10004500
user 4600: 10004600
user 4700: 10004700
user 4800: 10004800
user 4900: 10004900
user 5000: 1

In [12]:
list(all_transactions.keys())[0]

10000000

In [ ]:
'chid', 'dt', 'shop_tag', 'txn_cnt', 'txn_amt'

In [21]:
all_transactions[10000000][3]

[[10000000,
  4,
  '10',
  2,
  8121.978476954087,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0],
 [10000000,
  4,
  '12',
  1,
  5382.251951097488,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0],
 [10000000,
  4,
  '15',
  1,
  4684.483371984783,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0],
 [10000000,
  4,
  '37',
  1,
  4235.791755461504,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0]]

In [ ]:
with open('./dataset/all_transaction_v4.pkl', 'wb') as f:
    pickle.dump(all_transactions, f)

In [ ]:
list(all_transactions.keys())[0]

In [ ]:
all_transactions[10000000]

## User Embedding

use user information to build a autoencoder or VAE

In [ ]:
a = torch.rand(16, 6, 50)

In [ ]:
b = torch.rand(16, 6, 50)

In [ ]:
c = torch.cat((a, b), dim=1)
c.shape

In [ ]:
x = torch.rand(100, 30, 50)
y = torch.rand(100, 5)

In [ ]:
torch.cat((a, b), dim=0).shape

In [ ]:
c = a[['chid', 'masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam', 'gender_code', 'age', 'primary_card']]
c

In [ ]:
for i, (k, v) in enumerate(c.groupby('chid')):
    unique_row = v.apply(lambda col: col.unique())
    if len(unique_row) != 1:
        print(unique_row)

In [ ]:
np.random.rand(4, 5, 1, 1).squeeze().shape

In [ ]:
v['masts'].unique()

In [ ]:
[0, 0, 0, 0, 0, 0, 0, 0]
       x, x, x
    
    x x

In [ ]:
a = np.random.rand(3, 5)

a

In [ ]:
a.shape

In [ ]:
np.expand_dims(a, axis=-1)

In [ ]:
a = a.reshape(15, 1)
a

In [ ]:
b = a.reshape(5, 3)
b

In [ ]:
b['masts'].isnull().values.sum()

In [ ]:
b['masts'].max()

In [ ]:
b['masts'].unique()

In [ ]:
c['educd'].unique()

In [ ]:
c['trdtp'].unique()

In [ ]:
c['naty'].unique()

In [ ]:
c['poscd'].unique()

In [ ]:
c['cuorg'].unique()

In [ ]:
c['age'].unique()

In [ ]:
b['gender_code'].unique()

In [ ]:
c['primary_card'].unique()